# Controlled Text Generation

**By:** Agustinus Kristiadi and Yonathan Santosa

**Reference:** Hu, Zhiting, et al. "Toward controlled generation of text." ICML 2017.

## Motivation
Deep generative models has been very popular recently in both theory and application due to its versatility. They have been successfully applied for image-to-image translation, image super-resolution, and style transfer, and more.

<img src="figs/pix2pix.png" style="width: 600px;"/>

Two of the most popular flavours of deep generative models are Generative Adversarial Networks (GAN) and Variational Autoencoder (VAE). They are approaching the problem of generative modeling from different perspectives and methods. However, recently there is a work that show the unified view of GAN and VAE. GAN approach the problem of generative modeling from game theoritic point of view, that is, finding the optimal approximate data distribution is equivalent to finding Nash Equilibrium. Meanwhile, VAE approaches generative modeling from statistical point of view, with the principle of Variational Inference that comes from statistical physics.

GAN and VAE has been very popular in recent years in vision domain. This popularity mainly attributed to the fact that GAN and VAE are readily available to be applied in continuous domain like images. Many works have been released in vision domain, e.g.. However, GAN and VAE have been sparsely used for problems in text domain. This underusage is attributed to the fact that text is discrete in its nature, and thus harder to be applied with GAN or VAE, as they are both rely on backpropagation which requires gradients of their parameters, which only available in continuous function.

Nevertheless, there have been some effort to incorporate GAN and VAE for textual data. In this project, we are implementing a model called _Conditional Generation of Text (CGT)_ by Hu, et. al., 2017, in which GAN and VAE are combined together to enable text generation conditioned to some labels, e.g. sentiment.

## Background

### Generative Models
Generative modeling is density estimation problem, i.e. given data samples or empirical distribution, we are interested to find the true distribution of those samples. This is often intractable task, thus we need some sort of approximation. We can do this by using maximum likelihood principle which says that the best approximated distribution is the one that most likely generates these data.

<img src="figs/mle.pdf" style="width: 300px;"/>

Given a generative model, as it estimates the $p(\mathcal{X})$, i.e. the true data distribution, by sampling from it, we can get new data. This motivates the application of generative models for semi-supervised learning.

In this section we will review the high level idea of GAN and VAE to provide necessary background for this report. We refer interested readers into Goodfellow, 2016 for detailed survey about generative modeling and especially GAN.

#### Generative Adversarial Networks (GAN)

Generative Adversarial Networks (GAN) is a generative models that uses approach from game theory. The basic idea of GAN is to have two players game where those players are neural networks. First network is called the \b{generator} and the second network is called the \b{discriminator}. The discriminator's job is to examine data samples, whether they are fake or real. The generator's job meanwhile is to fool the discriminator, i.e. generates new samples that are very realistic so that discriminator misclassify them as real. Drawing analogy from money counterfeiter (generator) and police (discriminator), the counterfeiter has to fool the police by creating fake money that are indistinguishible from the real money. Meanwhile the police's job is to make sure that counterfeited money will not circulated in public, thus the police needs to be able to do good job in classifying between real and fake money.

Formally, GAN consists of two functions: discriminator function $D$ and generator function $G$. $D$ takes data $\mathbf{x}$ as inputs and has parameters $\mathbf{\theta^D}$. The generator function $G$ takes latent variables $\mathbf{z}$ as inputs and has parameters $\mathbf{\theta^G}$. In practice both $D$ and $G$ are both deep neural networks.

Both players in GAN has their own objective, denoted in their cost functions. The discriminator's job is to minimize $J^D(\mathbf{\theta^D}, \mathbf{\theta^G})$ while only modifying $\mathbf{\theta^D}$. Likewise, the generator's job is to minimize $J^G(\mathbf{\theta^D}, \mathbf{\theta^G})$ while only modifying $\mathbf{\theta^G}$. Thus, each players cannot control behavior of other player. The solution of this optimization problem is then a Nash Equilibrium. The overall cost function of this game is as follows:

\begin{equation}
  \min_G \max_D V(D, G) = \mathbb{E}_{x \sim p_{\text{data}}(x)}[\log D(x)] + \mathbb{E}_{z \sim p_{z}(z)}[\log (1 - D(G(z)))].
\end{equation}

#### Variational Autoencoder (VAE)

Variational Autoencoder (VAE) is another class of approaching the problem of generative modeling. Unlike GAN which uses game theoritic approach, VAE borrows its idea from statistical physics. Concretely, VAE uses mean-field theorem to formulate mean-field variational inference~\cite{blei2017variational} for generative modeling.

Like GAN, VAE consists of two parametric functions, which in this case are deep neural networks. The first network is the recognition or inference network $Q(z \vert x)$ which takes data $x$ as inputs and maps them into the latent variables $z$. The second network is the generative network $P(x \vert z)$ which takes latent variables as inputs and generates data. Notice that the formulation of VAE is very similar to classical autoencoder (AE), hence the name.

VAE differs from AE in the way the hidden or latent variables $z$ is represented. In VAE $z$ are stochastic variables, and the hidden units of $z$ are encoded by a distribution. In practice, this distribution is diagonal Gaussian.

The overall cost function of VAE is the combination of reconstruction cost and hidden units cost. Reconstruction cost in VAE is analogue to that in AE. The hidden units cost, which minimize KL-Divergence between posterior $Q(z \vert x)$ and prior $P(z)$ can be seen as a regularization for the hidden units of the autoencoder. Thus the total cost of VAE is as follows:

\begin{equation}
  \log P(x) \geq \mathbb{E}[\log P(x \vert z)] - D_{KL}[Q(z \vert x) \Vert P(z)] .
\end{equation}

That is, VAE circumvents the intractability of $P(\mathcal{X})$ by optimizing its lower bound instead. The name of variational comes from the fact that this lower bound comes by applying mean-field variational inference. This lower bound will not exactly fit the target distribution, but we hope that it will be a good approximation of it.

<img src="figs/main-figure0.pdf" style="width: 400px;"/>

### Generative Models for Text

Generative modeling, unlike in vision domain, is largely still in infancy. This is due to the nature of textual data, i.e. texts are discrete and in Deep Learning, we prefer everything to be continuous as it is easier to optimize.

To see the difficulty of generative modeling on text, consider a GAN. Consider GAN loss defined above, we can see that the output of generator $G$, which in this case text, will be fed into $D$. During the backpropagation then, to get the gradient of $G$, we have to backpropagate from the output of $D$ to $G$ via the output of $G$. As this output is discrete, then the gradient is undefined and thus breaks backpropagation.

However, there are some tricks that can be used to avoid this pitfall. For example, we can use softmax trick using temperature to do continuous approximation of argmax or even considering this problem from Reinforcement Learning perspective.


## Project Overview

### Architecture

<img src="figs/arch.pdf" style="width: 400px;"/>

### Dataset and Framework
We use PyTorch 0.3 to implement with SST-datasets to train the Controlled Generation of Text (CGT). This dataset is a sentiment dataset. We filtered the sentences with sentence length $\leq 15$ resulting $2837$ training samples.

### Training Overview

In [ ]:
Input = Corpus with labels {(x_L, c_L)}

Initialize base VAE by minimizing L_VAE on x_L with c sampled from prior p(c)

repeat:
    Train discriminator by minimizing L_D
    Train decoder by minimizing L_G
    Train encoder by minimizing L_VAE
until converge

### Training Base VAE (Bowman, 2016)

#### Embedding Vector 
Here the input sentence will be transformed into word embedding. A word embedding is a mapping of the word vocabulary with the size $v$ into a n-dimensional for $\mathbb{R}^n$. Each word has a representation of embedding word and collection of the embedding word called embedding vector. For this project, we use a pretrained model Global Vector for Word Representation (GloVe) (Pennington et al., 2014) with $n=50$. This embedding vector was trained on Wikipedia 2014 and Gigaword 5 datasets.

#### Encoder Model
The encoder model consist of a Gated Recurrent Unit (GRU) and an Multi Layer Perceptron. First we transformed the input sentence into an embedding vector then we fed these embedding vector into GRU. We use the last context layer (hidden state) of GRU to be fed into MLP resulting $\mu$ and $\log \sigma^2$. These parameters is later used on getting the laten code $z$

In [ ]:
self.encoder = nn.GRU(self.emb_dim, h_dim)
self.q_mu = nn.Linear(h_dim, z_dim)
self.q_logvar = nn.Linear(h_dim, z_dim)

z is sampled from $\mathcal{N}(\mu, \sigma^2)$ with a reparameterization tricks defined as follows:

\begin{align}
\epsilon &\sim \mathcal{N}(0,I)\\
z &= \mu + \sigma \cdot \epsilon\\
  &= \mu + exp\left(\frac{\log \sigma^2}{2}\right) \cdot \epsilon
\end{align}

In [ ]:
def sample_z(self, mu, logvar):
    """
    Reparameterization trick: z = mu + std*eps; eps ~ N(0, I)
    """
    eps = Variable(torch.randn(self.z_dim))
    eps = eps.cuda() if self.gpu else eps
    return mu + torch.exp(logvar/2) * eps

#### Decoder Model
The decoder model is a GRU that takes a latent variable $z$ and another structured code $c$ as the initial value for the context layer. $c$ is taken from a prior distribution $p(c)$. The decoder takes a starting token $\text{<START>}$ as the input value.

In [ ]:
def forward_decoder(self, inputs, z, c):
    """
    Inputs must be embeddings: seq_len x mbsize
    """
    dec_inputs = self.word_dropout(inputs)

    # Forward
    seq_len = dec_inputs.size(0)

    # 1 x mbsize x (z_dim+c_dim)
    init_h = torch.cat([z.unsqueeze(0), c.unsqueeze(0)], dim=2)
    inputs_emb = self.word_emb(dec_inputs)  # seq_len x mbsize x emb_dim
    inputs_emb = torch.cat([inputs_emb, init_h.repeat(seq_len, 1, 1)], 2)

    outputs, _ = self.decoder(inputs_emb, init_h)
    seq_len, mbsize, _ = outputs.size()

    outputs = outputs.view(seq_len*mbsize, -1)
    y = self.decoder_fc(outputs)
    y = y.view(seq_len, mbsize, self.n_vocab)

    return y

The decoder outputs $o$ then fed into a fully connected layer with activation softmax
$$
\hat{x} \sim softmax\left(\frac{o}{\tau}\right)
$$
where $\tau$ is the temperature of the softmax set to $\tau \rightarrow 0$ as the training proceeds.

#### Loss Function
To train the base VAE, we define the objective function as follow:

$$
L_{VAE} = \mathbb{E}_{q_E,q_D}\left[\log p_G(x \mid z,c)\right]
-KL(q_E(z \mid x) \Vert p(z))
$$

Basically we can read $L_{VAE}$ as combination of reconstruction loss and normalization term. In this project we call generator and decoder interchangebly thus in the previous calculation, $q_E$ denotes the encoder, $p_G$ denotes the generator, and $q_D$ denotes discriminator. What is discriminator and how it is used in this approach will be described shortly.

#### Optimization Detail

As in Bowman, 2016, we use two optimization tricks to make training stable and preventing KL term collapses. Those tricks are KL weight annealing and word dropout.

KL weight annealing works by adding a weight to the KL term in the VAE loss and we anneal the value from 0 to 1 as the optimization progresses. The snippet of the code is as follows:

In [ ]:
# Annealing for KL term
kld_start_inc = 3000
kld_weight = 0.01
kld_max = 0.15
kld_inc = (kld_max - kld_weight) / (n_iter - kld_start_inc)

recon_loss, kl_loss = model.forward(inputs)
loss = recon_loss + kld_weight * kl_loss

# Anneal kl_weight
if it > kld_start_inc and kld_weight < kld_max:
    kld_weight += kld_inc

This scheme is useful to ensure that KL term does not go to zero as that means that we do not learn anything useful.

<img src="figs/kl_weight_chart.pdf" style="width: 400px;"/>

The second trick we use is word dropout, which is used to regularize the decoder. It works by randomly set decoder input to $\text{"<UNK>"}$ with probability $p$. It is implemented as follows:

In [ ]:
def word_dropout(self, inputs):
    """
    Do word dropout: with prob `p_word_dropout`, set the word to '<unk>'.
    """
    if isinstance(inputs, Variable):
        data = inputs.data.clone()
    else:
        data = inputs.clone()

    # Sample masks: elems with val 1 will be set to <unk>
    mask = torch.from_numpy(
        np.random.binomial(1, p=self.p_word_dropout, size=tuple(data.size()))
                 .astype('uint8')
    )

    if self.gpu:
        mask = mask.cuda()

    # Set to <unk>
    data[mask] = self.UNK_IDX

    return Variable(data)

### Training CGT
Now that we have the base VAE to generate sentences, we would like to enforce the constraint on the structured code $c$. That is, we would like to make the VAE to only generate sentence with given code $c$. In order to do that, we have to train the Discriminator together with the Encoder and Decoder. The Discriminator is a ConvNet (Kim, 2014) that takes the word embeddings of a sentence and produce a probability $p(\hat{c})$.

In this section we will look at how training objective for Encoder, Decoder, and Discriminator are defined.

#### Encoder
The encoder loss is minimized by using $L_{VAE}$ described earlier

#### Decoder
The decoder loss then reformulated to enforce the network to train on the latent variable $z$ and the structured code $c$ while minimizing the reconstruction error at the same time. The loss is defined as follows:

\begin{align}
L_{Attr,z}(\theta_G) &= \mathbb{E}_{p(z)p(c)}\left[\log q_E(z\mid \tilde{G}_{\tau}(z,c)\right]\\
L_{Attr,c}(\theta_G) &= \mathbb{E}_{p(z)p(c)}\left[\log q_D(c\mid \tilde{G}_{\tau}(z,c)\right]\\
L_G &= L_{VAE} + \lambda_c L_{Attr,c} + \lambda_z L_{Attr,z}
\end{align}

$\tilde{G}_{\tau}(z,c)$ denotes a "soft embedding of generated sentence". The reason for this is that we cannot use $argmax$ to choose one of the words of the output represented by $\hat{x}$ since $argmax$ function is not differentiable which imply that the gradient cannot be back-propagated. Thus the following approach is suggested.

Let $\mathbf{E}$ be the embedding matrix, then the soft embedding at each time step of the decoder is given by:

$$
   \tilde{G}_{\tau}(z,c) = \mathbb{E}_{p_{G_{\tau}}(x \vert z, c)}[\mathbf{E}]
$$

First we sample $z$ and $c$ from a prior distribution $p(z)$ and $p(c)$ respectively. Then we use this $z$ and $c$ to generate a sentence with the length $K$ by using the decoder. At each time step, we get a probability distribution over words in our vocabulary. We use this distribution to take the expected embedding. Note that instead of $argmax$, we take the expectation instead and by doing so, we solve the nondifferentiability problem in our Discriminator. The snippet to generate this soft embedding is as follows:

In [ ]:
def sample_soft_embed(self, z, c, temp=1):
    """
    Sample single soft embedded sentence from p(x|z,c) and temperature.
    Soft embeddings are calculated as weighted average of word_emb
    according to p(x|z,c).
    """
    self.eval()

    z, c = z.view(1, 1, -1), c.view(1, 1, -1)

    word = torch.LongTensor([self.START_IDX])
    word = word.cuda() if self.gpu else word
    word = Variable(word)  # '<start>'
    emb = self.word_emb(word).view(1, 1, -1)
    emb = torch.cat([emb, z, c], 2)

    h = torch.cat([z, c], dim=2)

    if not isinstance(h, Variable):
        h = Variable(h)

    outputs = [self.word_emb(word).view(1, -1)]

    for i in range(self.MAX_SENT_LEN):
        output, h = self.decoder(emb, h)
        o = self.decoder_fc(output).view(-1)

        # Sample softmax with temperature
        y = F.softmax(o / temp, dim=0)

        # Take expectation of embedding given output prob -> soft embedding
        # w * y = n_vocab x emb_dim * n_vocab x 1
        emb = (self.word_emb.weight * y.unsqueeze(1)).sum(0).view(1, 1, -1)

        # Save resulting soft embedding
        outputs.append(emb.view(1, -1))

        # Append with z and c for the next input
        emb = torch.cat([emb, z, c], 2)

    # 1 x 16 x emb_dim
    outputs = torch.cat(outputs, dim=0).unsqueeze(0)

    # Back to default state: train
    self.train()

    return outputs.cuda() if self.gpu else outputs

#### Discriminator
The discriminator contains of Convolutional Neural Network (CNN) and a fully connected layer from the last conv layer to produce sentiment probability $p(\hat{c})$. Let a sentence $\mathbf{x}= [x_1, x_2, ..., x_n]$ be the input sentence with sequence length of $n$, we translate each $x_i$, for $x_i \in \mathbf{x}$, into word embeddings with embedding dimension $m$ and stack them together as matrix $\mathbf{E_x}$ 
depicted as follows:

$$\mathbf{E_x} =
\begin{bmatrix}
\mathbf{e}_{x_1} \\
\mathbf{e}_{x_2} \\
\vdots \\
\mathbf{e}_{x_n}
\end{bmatrix}
=
\begin{bmatrix}
e_{x_1}^1 & e_{x_1}^2 & \dotsb & e_{x_1}^m \\
e_{x_2}^1 & e_{x_2}^2 & \dotsb & e_{x_2}^m \\
\vdots & \vdots & \ddots & \vdots\\
e_{x_n}^1 & e_{x_n}^2 & \dotsb & e_{x_n}^m \\
\end{bmatrix}
$$

thus for each convolutional layer, if we do convolution on $\mathbf{E_x}$ with kernel size $[3 \times embedding\,dimension]$, it is analogous with tri-gram. In this case we do it with kernel size $[\{3,4,5\} \times embedding\,dimension]$. The architecture is as follows:

<img src="figs/kim_cnn.png" style="width: 700px;"/>

It is implemented in our model as follows:

In [ ]:
"""
Discriminator is CNN as in Kim, 2014
"""
self.conv3 = nn.Conv2d(1, 100, (3, self.emb_dim))
self.conv4 = nn.Conv2d(1, 100, (4, self.emb_dim))
self.conv5 = nn.Conv2d(1, 100, (5, self.emb_dim))

self.disc_fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(300, 2)
)

def forward_discriminator_embed(self, inputs):
    """
    Inputs must be embeddings: mbsize x seq_len x emb_dim
    """
    inputs = inputs.unsqueeze(1)  # mbsize x 1 x seq_len x emb_dim

    x3 = F.relu(self.conv3(inputs)).squeeze()
    x4 = F.relu(self.conv4(inputs)).squeeze()
    x5 = F.relu(self.conv5(inputs)).squeeze()

    # Max-over-time-pool
    x3 = F.max_pool1d(x3, x3.size(2)).squeeze()
    x4 = F.max_pool1d(x4, x4.size(2)).squeeze()
    x5 = F.max_pool1d(x5, x5.size(2)).squeeze()

    x = torch.cat([x3, x4, x5], dim=1)

    y = self.disc_fc(x)

    return y

To compute the discriminator loss, we use two sets of sentences taken from the labeled training data ${(x_L, c_L)}$ and fake sentences $(\hat{x}, \hat{c})$ produced from the decoder $p_G$. The loss is defined as follows:

\begin{align}
L_s(\theta_D) &= \mathbb{E}_{x_L}\left[\log q_D(c_L\mid x_L)\right]\\
L_u(\theta_D) &= \mathbb{E}_{p_G(\hat{x}\mid x,c)p(z)p(c)}\left[\log q_D(c\mid\hat{x}) + \beta \mathcal{H}(q_D(c'\mid\hat{x}))\right]\\
L_D &= L_s + \lambda_u L_u
\end{align}

Where $\mathcal{H}$ is empirical Shannon entropy of distribution $q_D$ evaluated on the sentence $\hat{x}$ and $\beta$ is a balancing parameter.

## Implementation Summary

To recap all of the above training ingredients, we show the full training loop below.

In [ ]:
for it in tqdm(range(n_iter)):
    inputs, labels = dataset.next_batch(args.gpu)

    """ Update discriminator, eq. 11 """
    batch_size = inputs.size(1)

    x_gen = model.generate_sentences(batch_size)  # mbsize x 16

    y_disc_real = model.forward_discriminator(inputs.transpose(0, 1))
    y_disc_fake = model.forward_discriminator(x_gen)

    log_y_disc_fake = F.log_softmax(y_disc_fake, dim=1)
    entropy = log_y_disc_fake.mean()

    loss_s = F.cross_entropy(y_disc_real, labels)
    loss_u = F.cross_entropy(y_disc_fake, labels) + beta*entropy

    loss_D = loss_s + lambda_u*loss_u

    loss_D.backward()
    grad_norm = torch.nn.utils.clip_grad_norm(model.discriminator_params, 5)
    trainer_D.step()
    trainer_D.zero_grad()

    """ Update generator, eq. 8 """
    # Forward VAE with c ~ q(c|x) instead of from prior
    recon_loss, kl_loss = model.forward(inputs, use_c_prior=False)
    # x_gen: mbsize x seq_len x emb_dim
    x_gen_attr, target_z, target_c = model.generate_soft_embed(batch_size, temp=temp(it))

    # y_z: mbsize x z_dim
    y_z, _ = model.forward_encoder_embed(x_gen_attr.transpose(0, 1))
    y_c = model.forward_discriminator_embed(x_gen_attr)

    loss_vae = recon_loss + kl_weight_max * kl_loss
    loss_attr_c = F.cross_entropy(y_c, target_c)
    loss_attr_z = F.mse_loss(y_z, target_z)

    loss_G = loss_vae + lambda_c*loss_attr_c + lambda_z*loss_attr_z

    loss_G.backward()
    grad_norm = torch.nn.utils.clip_grad_norm(model.decoder_params, 5)
    trainer_G.step()
    trainer_G.zero_grad()

    """ Update encoder, eq. 4 """
    recon_loss, kl_loss = model.forward(inputs, use_c_prior=False)

    loss_E = recon_loss + kl_weight_max * kl_loss

    loss_E.backward()
    grad_norm = torch.nn.utils.clip_grad_norm(model.encoder_params, 5)
    trainer_E.step()
    trainer_E.zero_grad()

## Result

The results of this project can be generated by running script `test.py`. It will generate results for conditional sentence generation and sentence interpolation. Below we show some examples.

### Conditional Generation

#### Positive sentiment, varying latent variable z
* the is full of the year 's best .
* a deftly entertaining thriller .
* a lovably movie .

#### Negative sentiment, varying latent variable z
* the actors are no chemistry or unappealing to care one . 
* delivers a living pretty bad movie .
* full of flatulence jokes and cliched to cliches .

### Sentence Interpolation

* **the is the courage of the unsalvageability of the year .**
* the is full of the year 's best .
* the 's unhurried pace is a lot of nada .
* the 's unhurried pace is repulsive and unfocused .
* **the 's unhurried pace is actually one of anything .**

## Conclusion and Discussion

In this project we worked on conditional generation of text, following a paper by Hu, 2017. With this model, we combine together two of the most popular generative models: GAN and VAE. The base VAE model is following a model by Bowman, 2016, and we put a discriminator on top of that to enforce conditional label in the text generation. We are able to train this model by using some tricks, namely KL-cost annealing, word dropout, and also soft embedding. With these tricks, training for base VAE is stabilized and the problem with nondifferentiability of textual data is alleviated when training the discriminator.

## References

1. Bowman, Samuel R., et al. "Generating Sentences from a
Continuous Space." CoNLL 2016.
2. Hu, Zhiting, et al. "Toward controlled generation of text."
ICML 2017.